In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import time
import random
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import requests
import sys
import json
import pickle
from itertools import combinations

# 네이버 기사 크롤링

In [2]:
def get_news(n_url): 
    news_detail = []
    news_title=[]
    headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
    'accept' : "*/*",
    'accept-encoding' : 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'referer' : n_url
    }
    
    payload = {'as_epq': 'James Clark', 'tbs':'cdr:1,cd_min:01/01/2015,cd_max:01/01/2015', 'tbm':'nws'}
    breq = requests.get(n_url, params=payload, headers=headers) 
    bsoup = BeautifulSoup(breq.content, 'html.parser') 
    try:
        title = bsoup.select('h3#articleTitle')[0].text 
        pdate = bsoup.select('.t11')[0].get_text()[:11] 
        news_detail.append(pdate) 
        _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ") 
        btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
        news_detail.append(btext.strip()) 
        pcompany = bsoup.select('#footer address')[0].a.get_text() 
        news_detail.append(pcompany) 

        return title, news_detail
    
    except:
        print("양식 다름")
        return [],[]


In [3]:
def get_news_naver(keyw, num_page):
    article_dict={"titles":[], "contents":[],"urls":[], "comments":[]}
    keyw = keyw.replace(' ', '+') 
    for keyword in [keyw]:#,"PM"
        for page in range(num_page):  # 페이지 개수 설정 1~400
            raw = requests.get(f'https://search.naver.com/search.naver?&where=news&query={keyword}&start=' + str(page * 10 + 1)).text
            soup = BeautifulSoup(raw, 'html.parser')
            articles = soup.select('#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li')

            for article in tqdm(articles):
                c_tag = article.select_one('div > div.info_group > a:nth-of-type(2)')
                comment_list=[]
                if c_tag!=None:
                    content= c_tag["href"]
                    article_dict["titles"].append(get_news(content)[0])
                    article_dict["contents"].append([get_news(content)[1][1]])
                    article_dict["urls"].append(content)
                    oid = content.split("oid=")[1].split("&")[0] 
                    aid = content.split("aid=")[1] 
                    page = 1
                    headers = {
                        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
                        'accept' : "*/*",
                        'accept-encoding' : 'gzip, deflate, br',
                        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
                        'referer' : content
                    }

                    while True:
                        c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(page) + "&refresh=false&sort=FAVORITE"
                        # 파싱하는 단계입니다.
                        r = requests.get(c_url, headers=headers)
                        cont = BeautifulSoup(r.content, "html.parser")
                        total_comm = str(cont).split('comment":')[1].split(",")[0]
                        match = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
                        # 댓글을 리스트에 중첩합니다.
                        comment_list.append(match)
                        # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
                        time.sleep(random.uniform(0.1,0.2))   # 2~4초 사이 랜덤한 시간으로  쉬어줘
                        if int(total_comm) <= ((page) * 20):
                            break
                        else:
                            page += 1
                    article_dict["comments"].append(comment_list)
    return article_dict

In [4]:
#naver_dict = get_news_naver('전동 킥보드', 1)

# 다음 기사 크롤링

In [5]:
def daum_search(keyword, num_page):
    keyword = keyword.replace(' ', '+') 
    url = 'https://search.daum.net/search?w=news&DA=PGD&spacing=0'
    daum_url_list = []
    for page in range(1, 1+num_page):
        req_params = {
            'q': keyword, # 검색어(키워드)를 쿼리 스트링에 파라미터로 추가
            'p': page # 검색 페이지 번호를 쿼리 스트링에 파라미터로 추가
        }
        response = requests.get(url, params=req_params)
        html = response.text.strip()

        soup = BeautifulSoup(html, 'html5lib')
        li_list = soup.find_all('div',{'class' : 'wrap_cont'})
        nb_list = [li.find('a',{'class':'f_nb'}) for li in li_list if not li.find('a',{'class':'f_nb'}) == None]
        daum_url_list.append([url.get('href') for url in nb_list])
        #time.sleep(random.uniform(2,4)) 
    return daum_url_list

In [6]:
def get_news_daum(keyword, num_page):
    url_list = sum(daum_search(keyword, num_page),[])
    news_detail = {"titles":[], "contents":[],"urls":[], "comments":[]}
    comment_list = []
    #news_class = []
    for d_url in tqdm(url_list):
        headers = {
        'User-Agent':
        'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'accept' : "*/*",
        'accept-encoding' : 'gzip, deflate, br',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        }
        payload = {'as_epq': 'James Clark', 'tbs':'cdr:1,cd_min:01/01/2015,cd_max:01/01/2015', 'tbm':'nws'}
        try:
            breq = requests.get(d_url, params=payload, headers=headers) 
            bsoup = BeautifulSoup(breq.text)
            news_class = bsoup.find('span',{'class' : 'ir_wa'}).get_text()
            if news_class == '뉴스':
                news_title = bsoup.select_one('h3.tit_view').get_text()
                news_title.replace('<h3 class="tit_view" data-translation="true">', "") 
                news_title.replace('</h3>','')
                news_title.replace('[영상]','')
                news_contents = ''
                news_comments = []

                for p in bsoup.select('div#harmonyContainer p'):
                    news_contents += p.get_text()

                #access_token_id 반환
                org = d_url
                article_id = org.split("/")[-1]
                req = requests.get(org)
                soup = BeautifulSoup(req.content)
                data_client_id = soup.find('div',{'class':'alex-area'}).get('data-client-id')

                # authorization 값 반환
                headers['referer'] = org
                token_url = "https://alex.daum.net/oauth/token?grant_type=alex_credentials&client_id={}".format(data_client_id)
                req = requests.get(token_url, headers=headers)
                access_token = json.loads(req.content)['access_token']
                authorization = 'Bearer '+access_token

                # article - comment 연결 짓는 key값 반환
                headers['authorization'] = authorization
                post_url = """https://comment.daum.net/apis/v1/ui/single/main/@{}""".format(article_id)
                req = requests.get(post_url, headers = headers)
                soup = BeautifulSoup(req.content,'html.parser')
                post_id = json.loads(soup.text)['post']['id']

                count = len(comment_list)
                offset = 0

                while True:
                    request_url = "https://comment.daum.net/apis/v1/posts/{}/comments?parentId=0&offset={}&limit=100&sort="\
                    "RECOMMEND&isInitial=false&hasNext=true".format(post_id, offset)

                    req = requests.get(request_url, headers=headers)
                    soup = BeautifulSoup(req.content,'html.parser')
                    temp_json_list = json.loads(soup.text)
                    for temp_json in temp_json_list:
                        temp_json['org_url'] = org
                        try:
                            news_comments.append(temp_json['content'])
                        except:
                            news_comments.append([])
                    comment_list.extend(temp_json_list)
                    if len(temp_json_list) < 100:
                        break
                    else:
                        offset += 100
                        time.sleep(1)

                news_detail['titles'].append(news_title)
                news_detail['contents'].append([news_contents])
                news_detail['urls'].append(d_url)
                news_detail['comments'].append(news_comments)

                time.sleep(random.uniform(2,4)) 
        except:
            pass
    return news_detail

In [9]:
#daum_dict = get_news_daum('전동 킥보드', 1)

### 네이버, 다음 기사 크롤링(max(네이버)=400, max(다음)=80)

In [ ]:
naver_dict = get_news_naver('전동 킥보드', 400)
daum_dict=get_news_daum('전동 킥보드', 80)

# 크롤링 된 결과 확인 및 저장

In [10]:
# naver_dict

In [11]:
# daum_dict

In [12]:
# naver_dict['contents']

In [13]:
# daum_dict['contents']

In [2]:
# path = 'C:/Users/commend/Dropbox/2021_2/intern/nlp/'

In [21]:
# # 저장
# with open(path+'naver_dict.pickle', 'wb') as f:
#     pickle.dump(naver_dict, f, pickle.HIGHEST_PROTOCOL)

# with open(path+'daum_dict.pickle', 'wb') as f:
#     pickle.dump(daum_dict, f, pickle.HIGHEST_PROTOCOL)

In [30]:
# 읽기
with open('./naver_dict.pickle', 'rb') as f:
    naver_dict = pickle.load(f)
    
with open('./daum_dict.pickle', 'rb') as f:
    daum_dict = pickle.load(f)

In [31]:
len(naver_dict['titles'])

1896

In [32]:
len(daum_dict['titles'])

276

In [33]:
# naver_dict['contents']

In [34]:
# daum_dict['titles'][:10]

# 네이버 U 다음


In [35]:
naver_idx = [naver_dict['titles'].index(x) for x in naver_dict['titles'] if type(x) == str]
naver_headlines = [naver_dict['titles'][idx] for idx in naver_idx]

In [36]:
interaction = set(daum_dict['titles']) & set(naver_headlines)
daum_inter_idx = [daum_dict['titles'].index(inter) for inter in interaction]

In [37]:
keys = list(daum_dict.keys())

daum={"titles":[], "contents":[],"urls":[], "comments":[]}
daum[keys[1]] = [daum_dict['contents'][i][0] for i in range(len(daum_dict['contents'])) if i not in daum_inter_idx]

keys.remove(keys[1])

for key in keys:
    for i in range(len(daum_dict[keys[0]])):
        if i not in daum_inter_idx:
            daum[key].append(daum_dict[key][i])

In [38]:
keys = list(naver_dict.keys())
naver={"titles":[], "contents":[],"urls":[], "comments":[]}
for key in keys:
    for i in range(len(naver_dict[keys[0]])):
        if i in naver_idx:
            naver[key].append(naver_dict[key][i])

In [39]:
naver['contents'] = [x[1] for x in naver['contents']]

In [40]:
merged = naver
for key in keys:
    for i in range(len(daum[keys[0]])):
        merged[key].append(daum[key][i])

In [41]:
merged['idx'] = list(range(len(merged[keys[0]])))

In [42]:
all_contents = dict()
all_contents['idx'] = merged['idx']
all_contents['contents'] = merged['contents']

# 자연어 전처리

In [43]:
def refine(contents): 
    pure = re.sub('\[.*?\]', '', str(contents))   
    pure = re.sub('\【.*?\】', '', pure)
    pure = re.sub('\(.*?\)', '', pure)
    pure = re.sub("[가-힣]+\s+기자"," ", pure)
    
    pure = re.sub('이메일', "", pure)
    pure = re.sub('검색', "", pure)
    pure = re.sub('전화', "", pure)
    pure = re.sub("PM",'피엠',pure) 

    pure = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)",'',pure) 

    pure = re.sub('[a-zA-Z0-9]',' ',pure).strip()
    
    p = re.compile("\W+|\d+")
    content = p.sub(" ", pure)
    content = re.sub("피엠",'PM',content) 

    return str(content)

In [44]:
def remove_short_articles(txt_list, stnd):
    new_txt_dict = {'idx':[], 'contents':[]}
    for idx, txt in zip(range(len(txt_list)), txt_list):
        if len(txt.split(' ')) >= stnd:
            new_txt_dict['contents'].append(txt)
            new_txt_dict['idx'].append(idx)
    return new_txt_dict

In [45]:
def find_remove_duplication(txt_list, stnd1, stnd2):
    idx = range(len(txt_list))
    idx_comb = list(combinations(idx, 2))
    idx_duplicated = []
    dp_setsplit=[]
    
    for i in tqdm(idx):
        dp_setsplit.append(set(txt_list[i].split(' ')))

    for idx in tqdm(idx_comb):
        ratio = len(dp_setsplit[idx[0]] & dp_setsplit[idx[1]]) / ((len(dp_setsplit[idx[0]])+len(dp_setsplit[idx[1]]))/2)

        if stnd1 <= ratio <= stnd2:
#             idx_duplicated.append(idx[0])
            idx_duplicated.append(idx[1])
            
    return sorted(list(set(idx_duplicated)))

In [46]:
contents_index=all_contents['idx']

In [47]:
all_contents_refined = [refine(cont) for cont in all_contents['contents']]

In [48]:
all_contents_refined2 = remove_short_articles(all_contents_refined, 35)

In [49]:
dup_idx=find_remove_duplication(all_contents_refined2['contents'], 0.9, 1.0)

100%|████████████████████████████████████████████████████████████████████| 1101870/1101870 [00:07<00:00, 150917.59it/s]


In [31]:
contents_index = [x for x in contents_index if x not in dup_idx]

In [50]:
all_contents_refined_dup = {'idx':[], 'contents':[]}
for i in range(len(all_contents_refined2['contents'])):
    if i not in dup_idx:
        all_contents_refined_dup['idx'].append(all_contents_refined2['idx'][i])
        all_contents_refined_dup['contents'].append(all_contents_refined2['contents'][i])

In [52]:
len(all_contents_refined_dup['contents'])

1344

In [64]:
all_contents_refined=all_contents_refined_dup['contents']

In [26]:
# with open('./all_contents_refined_dup.pickle', 'wb') as f:
#     pickle.dump(all_contents_refined_dup, f)

In [27]:
# pickle로 저장해 둠

In [28]:
# path = 'C:/Users/commend/Dropbox/2021_2/intern/nlp/'

In [29]:
# with open('./all_contents_refined', 'rb') as f:
#     all_contents_refined = pickle.load(f)

## Comments, urls, titles

In [55]:
all_comments_refined = [x for x,idx in zip(merged['comments'],range(len(merged['comments']))) if idx in contents_index]
all_titles_refined = [x for x,idx in zip(merged['titles'],range(len(merged['titles']))) if idx in contents_index]
all_urls_refined = [x for x,idx in zip(merged['urls'],range(len(merged['urls']))) if idx in contents_index]

In [56]:
for i in range(10):
    print(all_titles_refined[i])
    print(all_urls_refined[i])
    print(all_contents_refined[i])
    print(all_comments_refined[i])


카카오T 앱으로 전동킥보드 '씽씽' 타보니
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002243470
저렴한 가격과 반납 지역 광범위 헬멧 운영 미비 판교 서비스 부재 등카카오모빌리티가 전동 킥보드 씽씽 연동 서비스를 지난 일부터 시작했다 카카오 앱에서 이용할 수 있다 회사는 그간 전기자전거 서비스 운영 경험을 살려 근래 이용 빈도가 높은 이동 수단인 전동 킥보드 사업에서도 경쟁력을 확보한다는 방침이다 내년 월 이전 지쿠터 도 연동할 예정이다 국내 카카오 이용자는 지난 월 누적 기준으로 천만 명가량 카카오모빌리티는 택시와 전기자전거 등 서비스를 통해 이동에 있어 일상생활에 편리함을 더했다 여기에 킥보드를 곁들였다 카카오 씽씽이 택시 전기자전거 서비스처럼 이용자 편의성을 제고해 서비스형모빌리티 플랫폼 선도 기업 명맥을 이어갈지 궁금했다 일 지디넷코리아는 직장인 밀집 지역을 중심으로 카카오 씽씽을 체험했다 씽씽은 서울 수원 용인 성남 부산 울산 광주 등에 만대 규모가 갖춰졌다 먼저 탑승지를 정했다 기자 거주지 지근거리인 경기 성남의 판교를 선택했다 정보기술 기업이 즐비해 최소 대 정도 있겠거니 생각했다 일 지하철 신분당선 판교역 번 출구 인근 카카오모빌리티가 서비스하는 씽씽 킥보드는 없었다 들뜬 마음으로 지하철 신분당선 판교역으로 향했다 카카오 앱을 켰다 앱 내 킥보드 페이지가 보이지 않았다 전기자전거 이용 카테고리인 바이크 를 눌러야 했다 첫 이용자라면 혼동할 만했다 이어 씽씽을 찾아 헤맸지만 한 대도 없었다 바이크에서 찾다 보니 전기자전거도 함께 나타나 헷갈렸다 네이버 본사가 있는 판교 인근 분당선 정자역 근처에서도 씽씽은 보이지 않았다 단 분당선 모란역과 가천대역 주변엔 수십 대 씽씽이 놓여있었다 직장인 유동 인구가 많은 서울 강남으로 이동했다 지하철 호선 강남역 주변엔 씽씽 외 스윙 빔 라임 다트 등 여러 킥보드가 나란히 서있었다 몇 걸음 지났을까 씽씽이 눈에

## stopwords & press

In [65]:
with open('./press.txt', "r", encoding='UTF-8') as file:
    strings = file.readlines()

In [66]:
press_ = [strg.split(',')[1:] for strg in strings]
press = [prs[1:] for prs in press_[0]]

In [67]:
none_dic = pd.read_csv('./none_dic.csv', header = None, encoding = "cp949")
none = none_dic[0].tolist()

## tokenization & bigram

In [68]:
from konlpy.tag import Komoran

# bigram 생성에 필요한 library
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# vectorize & lda에 필요한 library
from gensim import corpora
from gensim.models.ldamodel import LdaModel

# gensim의 ldamodel에 최적화된 라이브러리
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  

In [69]:
komo = Komoran(userdic='user_dic.txt')

In [70]:
def text_preprocessing(text,tokenizer):
    
    tokens = []
    stopwords = none 
    
    #token = tokenizer.morphs(text) # 모든 형태소 처리(morphs) 
    token = tokenizer.nouns(text) # 명사만 뽑기(nouns)
    
    token = [t for t in token if t not in stopwords] # 불용어 필터
    token = [t for t in token if t not in press if len(t) > 1]
    tokens.append(' '.join(token))
    
    return tokens

In [71]:
all_contents_nouns = []
for cont, idx in tqdm(zip(all_contents_refined, range(len(all_contents_refined)))):
    all_contents_nouns.append([x.split(' ') for x in text_preprocessing(cont, komo)][0])

1344it [00:27, 49.20it/s]


In [72]:
# for i in all_contents_nouns:
#     if len(i)<=1:
#         print(i)

In [73]:
all_contents_nouns_org = all_contents_nouns.copy()

In [74]:
# bigrams/trigrams를 더한다. (20번이 넘게 나타나는 것들만)
bigram = Phrases(all_contents_nouns, min_count=20)
for idx in range(len(all_contents_nouns)):
    for token in bigram[all_contents_nouns[idx]]:
        if '_' in token:
            # 토큰이 bigram이면 document에 추가한다.
            all_contents_nouns[idx].append(token)

In [75]:
dictionary = corpora.Dictionary(all_contents_nouns) 
# rare & common tokens 지우기
# 너무 자주 나오거나 너무 안나오는 단어들 지우기
max_freq = 0.6
min_wordcount = 2
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)
 
_ = dictionary[0]  # dictionary.id2token를 시작해주는 역할을 한다.

In [77]:
corpus = [dictionary.doc2bow(doc) for doc in all_contents_nouns]

In [78]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 6144
Number of documents: 1344


In [80]:
# corpus

## hyperparameter tuning

In [81]:
from tqdm import tqdm

In [84]:
num_topics_l = list(range(6,11))
chunksize_l = [100, 1000, 2000]
# passes_l = [10, 20, 30, 40]
# iterations_l = [100, 400, 500, 1000]
# random_state_l = [2021, 2022, 12, 30, 0]

# num_topics_l = [5]#list(range(3,11))
# chunksize_l = [2000]
passes_l = [40]#[20, 30, 40]
iterations_l = [500]
random_state_l = [2022]
# #random_state_l = [2021, 2022, 12, 30, 0]

model_list = []
                                                        
for random_state in random_state_l:                    
    for chunksize in tqdm(chunksize_l):
        for passes in passes_l:
            for iterations in iterations_l:
                for num_topics in num_topics_l:
                    model = LdaModel(corpus = corpus, id2word = dictionary, chunksize = chunksize,
                                           alpha ="auto", eta="auto",
                                           iterations = iterations, num_topics = num_topics,
                                           passes = passes, random_state=random_state)
                    top_topics = model.top_topics(corpus)
                    tc = sum([t[1] for t in top_topics])
                    hyper = [chunksize, passes, iterations, num_topics]
                    model_list.append((model, tc, hyper))
                    print('done')

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

done
done
done
done


 33%|███████████████████████████▋                                                       | 1/3 [01:44<03:29, 104.88s/it]

done
done
done
done
done


 67%|███████████████████████████████████████████████████████▎                           | 2/3 [04:18<02:13, 133.33s/it]

done
done
done
done
done


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [09:41<00:00, 193.97s/it]

done


In [356]:
with open('./hyper_list_0104.pickle', 'wb') as f:
    pickle.dump(hyper_list, f)

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [354]:
with open('./model_list_0104.pickle', 'wb') as f:
    pickle.dump(model_list, f)

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# grid search 결과 저장해 둠
with open(path + 'model_list_0103', 'rb') as f:
    result_list = pickle.load(f)

In [85]:
len(model_list)

15

In [86]:
result_list= model_list.copy()

In [87]:
model_list = [list(x) for x in result_list]
model_perplexity = [x[0].log_perplexity(corpus) for x in model_list]

In [88]:
results = pd.DataFrame(data={'chunksize':[x[2][0] for x in model_list],'condition_ntopic':[x[2][3] for x in model_list],'coherence':[x[1] for x in model_list], 'perplexity':model_perplexity})

In [89]:
results.sort_values(by='coherence', ascending=False)

,chunksize,condition_ntopic,coherence,perplexity
0,100,6,-8.321326,-7.055294
5,1000,6,-8.626258,-6.990012
10,2000,6,-8.896585,-6.976168
11,2000,7,-10.247469,-6.975178
12,2000,8,-12.318074,-6.959436
1,100,7,-13.029884,-7.087512
6,1000,7,-13.559048,-7.004783
7,1000,8,-13.563081,-6.987810
2,100,8,-13.575224,-7.093904
13,2000,9,-13.825182,-6.955502


In [90]:
results.sort_values(by='perplexity')

,chunksize,condition_ntopic,coherence,perplexity
4,100,10,-23.156763,-7.141399
2,100,8,-13.575224,-7.093904
3,100,9,-17.954764,-7.089160
1,100,7,-13.029884,-7.087512
0,100,6,-8.321326,-7.055294
6,1000,7,-13.559048,-7.004783
5,1000,6,-8.626258,-6.990012
7,1000,8,-13.563081,-6.987810
8,1000,9,-15.494408,-6.983310
10,2000,6,-8.896585,-6.976168


In [91]:
model_list

[[<gensim.models.ldamodel.LdaModel at 0x1da1b539a00>,
  -8.321325994554828,
  [100, 40, 500, 6]],
  -13.02988383609824,
  [100, 40, 500, 7]],
  -13.575223531308769,
  [100, 40, 500, 8]],
  -17.954763519381288,
  [100, 40, 500, 9]],
  -23.156762951596153,
  [100, 40, 500, 10]],
  -8.626257646670236,
  [1000, 40, 500, 6]],
  -13.559048398799575,
  [1000, 40, 500, 7]],
  -13.563081004471833,
  [1000, 40, 500, 8]],
  -15.494408026994119,
  [1000, 40, 500, 9]],
  -17.502912320875808,
  [1000, 40, 500, 10]],
  -8.896585384164513,
  [2000, 40, 500, 6]],
  -10.247469415737884,
  [2000, 40, 500, 7]],
  -12.31807443164966,
  [2000, 40, 500, 8]],
  -13.82518207137636,
  [2000, 40, 500, 9]],
  -15.997949752208704,
  [2000, 40, 500, 10]]]

## best model 시각화

In [92]:
model, tc = max([tuple(mod[:2]) for mod in model_list], key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

Topic coherence: -8.321e+00


In [93]:
model, tc, hyper = model_list[12]

In [94]:
model_list[12]

 -12.31807443164966,
 [2000, 40, 500, 8]]

In [95]:
print('\nPerplexity: ', model.log_perplexity(corpus))


Perplexity:  -6.959435931889623


In [206]:
# Save model.
model.save(path + '/model.atmodel')

In [33]:
# Load model.
model_saved = LdaModel.load(path + '/model.atmodel')

In [96]:
model.show_topic(3)

[('전동킥보드', 0.030940222),
 ('견인', 0.023484232),
 ('공유', 0.019944372),
 ('업체', 0.019242711),
 ('주차', 0.0177457),
 ('서울시', 0.015045817),
 ('안전', 0.014757436),
 ('이용자', 0.011050136),
 ('구역', 0.0107446015),
 ('불법', 0.010432074)]

In [97]:
model.print_topics(num_words=20) 

[(0,
  '0.025*"서비스" + 0.023*"공유" + 0.020*"모빌리티" + 0.011*"이용" + 0.010*"업체" + 0.010*"운영" + 0.009*"자전거" + 0.008*"사업" + 0.007*"대표" + 0.007*"시장" + 0.007*"국내" + 0.007*"이동" + 0.007*"전동킥보드" + 0.007*"이용자" + 0.006*"제공" + 0.006*"씽씽" + 0.006*"안전" + 0.006*"업계" + 0.006*"전기" + 0.006*"지역"'),
 (1,
  '0.018*"전동킥보드" + 0.017*"장치" + 0.017*"면허" + 0.017*"이동" + 0.015*"착용" + 0.015*"개인형" + 0.015*"개인형_이동" + 0.014*"이용" + 0.014*"도로교통법" + 0.012*"범칙금" + 0.012*"자전거" + 0.012*"헬멧" + 0.012*"운전" + 0.012*"안전" + 0.011*"안전모" + 0.011*"사고" + 0.011*"개정" + 0.011*"시행" + 0.010*"단속" + 0.010*"만원"'),
 (2,
  '0.035*"사고" + 0.029*"보험" + 0.025*"전동킥보드" + 0.016*"자동차" + 0.013*"주행" + 0.012*"경우" + 0.012*"가입" + 0.011*"과실" + 0.011*"보상" + 0.009*"뉴런" + 0.009*"운전자" + 0.009*"이용자" + 0.008*"안전" + 0.008*"PM" + 0.007*"보장" + 0.007*"발생" + 0.007*"교통사고" + 0.006*"차량" + 0.006*"상해" + 0.006*"위험"'),
 (3,
  '0.031*"전동킥보드" + 0.023*"견인" + 0.020*"공유" + 0.019*"업체" + 0.018*"주차" + 0.015*"서울시" + 0.015*"안전" + 0.011*"이용자" + 0.011*"구역" + 0.010*"불법" + 0.009*"방치" + 0.009*"

In [98]:
vis = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis)

In [99]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)


        for j, (topic_num, prop_topic) in enumerate(doc): 
            if j == 0:  
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
            else:
                break
    return(topic_table)

In [100]:
topictable = make_topictable_per_doc(model, corpus)
topictable = topictable.reset_index() 
topictable.columns = ['No_document', 'No_topic', 'similarity', 'detail']
topictable[:10]

,No_document,No_topic,similarity,detail
0,0,0.0,0.8822,"[(0, 0.88220906), (1, 0.11704515)]"
1,1,4.0,0.5902,"[(0, 0.2514812), (2, 0.1351492), (4, 0.5901751..."
2,2,7.0,0.7083,"[(1, 0.29002735), (7, 0.7082584)]"
3,3,2.0,0.7148,"[(1, 0.1659621), (2, 0.7147711), (5, 0.11745246)]"
4,4,0.0,0.5633,"[(0, 0.5633135), (1, 0.069724545), (3, 0.27943..."
5,5,1.0,0.6698,"[(1, 0.6698244), (7, 0.32671684)]"
6,6,6.0,0.3250,"[(2, 0.11734286), (4, 0.14939494), (5, 0.30802..."
7,7,2.0,0.5296,"[(0, 0.024645532), (2, 0.5295775), (6, 0.44171..."
8,8,0.0,0.7577,"[(0, 0.7576853), (1, 0.23298848)]"
9,9,2.0,0.9100,"[(2, 0.9099973), (5, 0.08819379)]"


In [103]:
high_simil = topictable.sort_values(by='similarity',ascending=False)

In [104]:
top2_high = [float(x) for x in range(0,6)]
top2_high*=10
save_top2=[]
for idx,rk in high_simil.iterrows():
    if rk['No_topic'] in top2_high:
        top2_high.remove(rk['No_topic'])
        save_top2.append([rk['No_topic'],rk['No_document'],rk['similarity']])
    if len(top2_high)==0:
        break
        

print(save_top2)

for i in save_top2:
    print(i[0])
    print(i[2])
    print(all_contents_refined[i[1]],end='\n\n')

[[3.0, 157, 0.9991000294685364], [1.0, 778, 0.9991000294685364], [1.0, 166, 0.9990000128746033], [2.0, 1242, 0.9990000128746033], [1.0, 678, 0.9990000128746033], [0.0, 965, 0.9988999962806702], [3.0, 1321, 0.9988999962806702], [3.0, 254, 0.9988999962806702], [1.0, 838, 0.9987999796867371], [1.0, 1118, 0.9987999796867371], [3.0, 312, 0.9987000226974487], [3.0, 602, 0.9986000061035156], [3.0, 22, 0.9986000061035156], [1.0, 71, 0.9986000061035156], [1.0, 854, 0.9986000061035156], [1.0, 1155, 0.9986000061035156], [0.0, 966, 0.9986000061035156], [3.0, 1342, 0.9986000061035156], [3.0, 261, 0.9984999895095825], [1.0, 753, 0.9984999895095825], [1.0, 238, 0.9984999895095825], [0.0, 1083, 0.9984999895095825], [0.0, 1105, 0.9983999729156494], [0.0, 563, 0.9983999729156494], [0.0, 203, 0.9983999729156494], [3.0, 1339, 0.9983000159263611], [3.0, 218, 0.998199999332428], [4.0, 216, 0.998199999332428], [2.0, 91, 0.998199999332428], [0.0, 1223, 0.998199999332428], [0.0, 378, 0.998199999332428], [4.0, 

In [532]:
top2_high = [float(x) for x in range(0,6)]
top2_high*=10
save_top2=[]
for idx,rk in high_simil.iterrows():
    if rk['No_topic'] in top2_high:
        top2_high.remove(rk['No_topic'])
        save_top2.append([rk['No_topic'],rk['No_document'],rk['similarity']])
    if len(top2_high)==0:
        break
        

print(save_top2)

for i in save_top2:
    print(i[0])
    print(i[2])
    print(all_contents_refined[i[1]],end='\n\n')

[[4.0, 45, 0.991100013256073], [4.0, 46, 0.9908000230789185], [3.0, 423, 0.9897000193595886], [4.0, 47, 0.9891999959945679], [0.0, 392, 0.9882000088691711], [0.0, 27, 0.9882000088691711], [5.0, 84, 0.9872000217437744], [1.0, 16, 0.9860000014305115], [3.0, 8, 0.9854999780654907], [3.0, 555, 0.9853000044822693], [5.0, 450, 0.9829999804496765], [4.0, 37, 0.982699990272522], [3.0, 9, 0.9825999736785889], [5.0, 169, 0.9787999987602234], [0.0, 6, 0.9776999950408936], [5.0, 82, 0.9757999777793884], [5.0, 499, 0.972100019454956], [3.0, 756, 0.9678000211715698], [5.0, 479, 0.9672999978065491], [5.0, 575, 0.9649999737739563], [0.0, 77, 0.9643999934196472], [3.0, 87, 0.9632999897003174], [3.0, 204, 0.9599999785423279], [5.0, 435, 0.9598000049591064], [4.0, 161, 0.9563000202178955], [1.0, 304, 0.9559000134468079], [0.0, 57, 0.9513999819755554], [3.0, 760, 0.9508000016212463], [3.0, 33, 0.9505000114440918], [0.0, 41, 0.9498000144958496], [1.0, 53, 0.939300000667572], [0.0, 63, 0.9354000091552734], 

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [534]:
top2_high = [float(x) for x in range(0,6)]
top2_high*=10
save_top2=[]
for idx,rk in high_simil.iterrows():
    if rk['No_topic'] in top2_high:
        top2_high.remove(rk['No_topic'])
        save_top2.append([rk['No_topic'],rk['No_document'],rk['similarity']])
    if len(top2_high)==0:
        break
        

print(save_top2)

for i in save_top2:
    print(i[0])
    print(i[2])
    print(all_contents_refined[i[1]],end='\n\n')

[[4.0, 137, 0.2409999966621399], [4.0, 672, 0.24660000205039978], [4.0, 18, 0.2583000063896179], [1.0, 194, 0.2596000134944916], [3.0, 607, 0.2635999917984009], [3.0, 677, 0.26499998569488525], [5.0, 245, 0.26739999651908875], [5.0, 753, 0.27129998803138733], [4.0, 605, 0.28220000863075256], [4.0, 434, 0.2896000146865845], [4.0, 455, 0.29120001196861267], [4.0, 638, 0.29159998893737793], [3.0, 454, 0.2922999858856201], [2.0, 490, 0.2939999997615814], [2.0, 259, 0.29739999771118164], [4.0, 120, 0.29829999804496765], [0.0, 209, 0.3001999855041504], [1.0, 703, 0.30059999227523804], [2.0, 568, 0.302700012922287], [1.0, 460, 0.3061000108718872], [3.0, 562, 0.30809998512268066], [4.0, 712, 0.30820000171661377], [4.0, 586, 0.3125999867916107], [0.0, 646, 0.31450000405311584], [0.0, 485, 0.3197000026702881], [1.0, 237, 0.3197999894618988], [3.0, 533, 0.3199000060558319], [0.0, 659, 0.32179999351501465], [3.0, 428, 0.32409998774528503], [1.0, 540, 0.32440000772476196], [2.0, 225, 0.324699997901

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
model.id2word()

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: 'Dictionary' object is not callable

In [116]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [117]:
token_list=[]
for cont in join_content:
    token_list.append(tokenizer.encode(cont, return_tensors="pt"))

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
token_all_sentence=[]
for idx in range(len(all_contents_refined)):
    token_all_sentence.append(tokenizer.encode(all_contents_refined[idx], return_tensors="pt"))

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [119]:
token_list

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[tensor([[    0, 18252, 14149,  9507, 14331, 25091, 18024, 10577, 14585, 14097,
          11028, 14274,  9160, 14756, 28686, 20887, 14069, 11033, 20658, 14038,
           9879,  1700, 13110, 25820,  1700, 11693, 11693, 27487, 14756, 14351,
          20887, 14533, 15454, 21123, 16465, 18796, 14756, 14585, 15211, 14356,
          10232, 14533, 15692,  9866, 15526, 25304, 14038,  9879,  1700, 13110,
          25820, 14379, 19920, 15206, 15627, 15220, 15752, 14034, 13015, 27487,
          14498, 14454, 20887, 27270, 21708, 14832, 20579, 20887, 14069, 11033,
          20658, 19469, 16465, 18796, 14756, 15526, 27430, 20200,  1700, 13110,
          25820, 20887,  1700, 11693, 11693, 19469, 16465, 18796, 14756, 27270,
          17506, 23865, 14756, 14069, 11033, 20658, 20052, 20138, 14413, 14248,
          10519, 20063, 26506, 27010, 14331, 14797, 20887,  1700, 11693, 11693,
          15959,  1700, 11693, 11693, 14245, 17250, 20340, 19106, 14802, 16176,
          15122, 14883, 23283, 18005, 12

In [120]:
list(token_all_sentence[0].size())[1]


C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1091

In [121]:
token_all_sentence[0]

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[    0, 12123, 10321,  ...,  9754,  1700,     1]])

In [129]:
test=token_all_sentence[0][:,:1026]

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
list(token_all_sentence[0][:,:1024].size())

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[1, 1024]

In [113]:
token_all_sentence[0]

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[    0, 12123, 10321,  ..., 19220, 10490, 16657]])

In [132]:
for sentence,idx in zip(token_all_sentence,range(len(token_all_sentence))):
    if list(sentence.size())[1]>1026:  #list(token_all_sentence[0].size())[1]
        token_all_sentence[idx]=sentence[:,:1026]

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [141]:
test= token_all_sentence[0:50]

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [142]:
test

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[tensor([[    0, 12123, 10321,  ..., 19220, 10490, 16657]]),
 tensor([[    0, 16176,  9776, 20550, 13587, 10948, 20731, 14060, 14183, 27389,
          28343, 26882, 11776, 14051, 12335, 15392, 17903, 22785, 16135, 16982,
          15707, 10518, 14280, 16176, 16765, 20550, 13587, 15225, 15628, 14051,
          12335, 22903, 22785, 16135, 14049, 14048, 19953, 18001, 14118, 15538,
          15313, 14055, 16176, 15132, 14655, 20550, 13587, 10948, 14789, 11465,
          11843, 14025,  9925, 13831, 17974, 15550, 14908, 14025, 19812, 15107,
          15509, 26594, 14934, 15131, 14788, 14441, 14739, 28343, 16935, 17360,
          11294, 15021, 18494, 17834, 14088, 12922, 13234, 17903, 22785, 15774,
           9754, 14025, 28784, 28343, 16935, 17360, 11294, 12007, 19756, 15527,
          14025, 10431, 12612, 23199, 19067, 14280, 28081, 12060, 15062, 11786,
          15501, 19085, 12124, 25165, 17834, 16508, 19430, 14189, 10325,  9754,
          16176,  9776, 26423, 14038,  9879,  1700, 13110, 

In [143]:
summary_text_ids=[]
for cont in tqdm(test):
    summary_text_ids.append(model.generate(
        input_ids=cont,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_length=128,
        min_length=32,
        num_beams=4
        )
    )

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:22<00:00,  2.85s/it]


In [147]:
summary_text_ids[0][0]

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([    2,     0, 25560, 20887, 10607, 11033, 20658,  9698, 14533, 15692,
        14973, 14904, 14038,  9879,  1700, 13110, 25820, 14379, 14590, 19911,
        15206, 14191, 14185, 14038,  9879,  1700, 13110, 25820,  1700, 11693,
        11693, 27487, 16657, 14141, 14055, 14150, 15358, 15844, 15220, 14147,
        14034, 13015, 13173,  9866, 14025, 27487, 13594, 26348, 14253, 14130,
            1])

In [150]:
tl=[]
for i in range(50):
    print(tokenizer.decode(summary_text_ids[i][0],skip_special_tokens=True),end="\n\n")
    tl.append(tokenizer.decode(summary_text_ids[i][0],skip_special_tokens=True))

지난 카카오모빌리티는 이용 빈도가 높은 전동 킥보드 사업에서도 경쟁력을 확보하기 위해 전동 킥보드 씽씽 연동 서비스를 지난 일부터 시행했고 내년에는 지쿠터도 이 연동할 계획이라고 밝혔다.

울산대 융합학부팀이 제 회 빅데이터 인공지능 스타트업 경진대회 에서 대상을 수상하고 급 증가하는 전동 킥보드 사고 예방을 위해 실시간으로 사용자의 헬멧 착용 여부를 감지하는 기능 주행 중인 노면의 경사도를 감지한 뒤 킥보드의 제한속도를 결정하는 기능을 개발하는 등 울산대 팀은 빅데이터 인공지능 관련 스타트업 및 예비 창업가를 지원하기 위해 준비된 경진대회 에서 각종 경진대회에서 대상을 수상하는 등 우수한 성과를 거두고 있다

공정거래위원회는 소비자에게 무면허로 운전하다 적발될 경우 만원의 범칙금이 부과된다는 내용을 알려야 하는 내용을 알려야 하는 전동 킥보드 판매 대여 업체는 무면허 운전을 포함해 음주 운전 동승자 탑승 안전모 미착용 보도 주행 세 미만 어린이의 운전 등의 내용을 알려야 한다는 내용을 포함한 중요한 표시 광고 사항 고시 를 일 개정해 시행한다고 밝혔다.

지난유튜브 채널 한문철 에는 지난 일 아이들 두 명이 탄 전동킥보드가 갑자기 나타나 사고가 났는데 촉법소년이라 처벌할 수도 없고 사고가 났는데 촉법소년이라 처벌할 수도 없고 부모가 합의금과 치료비를 대야 하는 상황이라는 제목의 영상이 올라왔는데 사고는 지난 일 오후 시경 수원의 한 골목에서 발생했다

전동킥보드를 비롯한 개인형 이동수단을 둘러싼 잡음이 계속되고 있다며, 해당 사업을 영위하는 업계 내에서도 을 바라보는 시각이 엇갈리고 업계 내에서도 서로 다른 의견에는 관련 시장의 급성장에 따른 부작용을 지목할 수 있다며, 전동킥보드 등 개인형 이동수단은 빠르게 성장해 제도권 이동수단으로 자리했다 제도권으로 들어온 만큼 이용자들의 안전과 공공의 이익을 고려하지 않는 업체는 빠르게 퇴출되도록 더욱 엄격한 제도가 마련돼야 한다며 킥보드 대수를 조절할 수 있고 사후관리에도 신경 쓰게 되야 사후관리를 제대로 하지 않거

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [170]:
a=' '.join(tl)

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [171]:
a

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'지난 카카오모빌리티는 이용 빈도가 높은 전동 킥보드 사업에서도 경쟁력을 확보하기 위해 전동 킥보드 씽씽 연동 서비스를 지난 일부터 시행했고 내년에는 지쿠터도 이 연동할 계획이라고 밝혔다. 울산대 융합학부팀이 제 회 빅데이터 인공지능 스타트업 경진대회 에서 대상을 수상하고 급 증가하는 전동 킥보드 사고 예방을 위해 실시간으로 사용자의 헬멧 착용 여부를 감지하는 기능 주행 중인 노면의 경사도를 감지한 뒤 킥보드의 제한속도를 결정하는 기능을 개발하는 등 울산대 팀은 빅데이터 인공지능 관련 스타트업 및 예비 창업가를 지원하기 위해 준비된 경진대회 에서 각종 경진대회에서 대상을 수상하는 등 우수한 성과를 거두고 있다 공정거래위원회는 소비자에게 무면허로 운전하다 적발될 경우 만원의 범칙금이 부과된다는 내용을 알려야 하는 내용을 알려야 하는 전동 킥보드 판매 대여 업체는 무면허 운전을 포함해 음주 운전 동승자 탑승 안전모 미착용 보도 주행 세 미만 어린이의 운전 등의 내용을 알려야 한다는 내용을 포함한 중요한 표시 광고 사항 고시 를 일 개정해 시행한다고 밝혔다. 지난유튜브 채널 한문철 에는 지난 일 아이들 두 명이 탄 전동킥보드가 갑자기 나타나 사고가 났는데 촉법소년이라 처벌할 수도 없고 사고가 났는데 촉법소년이라 처벌할 수도 없고 부모가 합의금과 치료비를 대야 하는 상황이라는 제목의 영상이 올라왔는데 사고는 지난 일 오후 시경 수원의 한 골목에서 발생했다 전동킥보드를 비롯한 개인형 이동수단을 둘러싼 잡음이 계속되고 있다며, 해당 사업을 영위하는 업계 내에서도 을 바라보는 시각이 엇갈리고 업계 내에서도 서로 다른 의견에는 관련 시장의 급성장에 따른 부작용을 지목할 수 있다며, 전동킥보드 등 개인형 이동수단은 빠르게 성장해 제도권 이동수단으로 자리했다 제도권으로 들어온 만큼 이용자들의 안전과 공공의 이익을 고려하지 않는 업체는 빠르게 퇴출되도록 더욱 엄격한 제도가 마련돼야 한다며 킥보드 대수를 조절할 수 있고 사후관리에도 신경 쓰게 되야 사후관리를 제대로 하지 않거나 위

In [172]:
total_summary=(model.generate(
    input_ids=a,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    length_penalty=2.0,
    max_length=128,
    min_length=32,
    num_beams=4
    )
)

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'str' object has no attribute 'shape'

In [45]:
join_content[0]

C:\Users\dogu\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'가격 반납 지역 광범위 헬멧 운영 미비 판교 서비스 부재 카카오 모빌리티 전동 킥보드 씽씽 연동 서비스 시작 카카오 이용 회사 그간 전기 자전거 서비스 운영 경험 근래 이용 빈도 이동 수단 전동 킥보드 사업 경쟁력 확보 방침 내년 이전 지쿠 연동 예정 국내 카카오 이용자 누적 기준 가량 카카오 모빌리티 택시 전기 자전거 서비스 이동 일상생활 편리 킥보드 카카오 씽씽 택시 전기 자전거 서비스 이용자 편의 제고 서비스 모빌리티 플랫폼 선도 기업 명맥 코리아 직장인 밀집 지역 중심 카카오 씽씽 체험 씽씽 서울 수원 용인 성남 부산 울산 광주 규모 탑승 거주지 근거리 경기 성남 판교 선택 정보기술 기업 최소 정도 생각 지하철 신분당선 판교역 출구 인근 카카오 모빌리티 서비스 씽씽 킥보드 마음 지하철 신분당선 판교역 카카오 킥보드 페이지 전기 자전거 이용 카테고리 바이크 이용자 라면 혼동 씽씽 대도 바이크 보니 전기 자전거 본사 판교 인근 분당선 정자역 근처 씽씽 분당선 모란역 가천대역 주변 씽씽 직장인 유동 인구 서울 강남 이동 지하철 호선 강남역 주변 씽씽 스윙 라임 다트 킥보드 걸음 씽씽 사용 절차 카카오 운전 면허증 인증 원동기 장치 자전거 면허 보유자 씽씽 사용 면허증 인증 사용 킥보드 코드 코드 킥보드 좌측 상단 킥보드 해제 이용 사용법 속도 킥보드 오른쪽 핸들 부착 가속 버튼 일시 정지 기능 이용 중단 좌측 핸들 안쪽 소리 경보 강남역 인근 지하철 호선 신논현역 차량 인적 도로 사용 방법 면허증 인증 대여 킥보드 코드 운행 시작 하면 된다 가량 배터리 요금 기본 요금 분당 추가 비용 발생 기본 요금 이용 합산 평일 주말 기본 요금 심야 이용 추가 요금 합리 가격 여타 전동 킥보드 가격 방식 킥보드 신논현역 강남역 이동 이용 가격 기본 요금 분당 부과 형태 킥보드 요금 지불 기준 킥보드 씽씽 천원 씽씽 반납 지역 범위 특장 반납 지역 한정 킥보드 씽씽 인근 대여 킥보드 안전 헬멧 구비 올해 도로교통법 개정 헬멧 착용 의무 착용 범칙금 만원 부과 이날 강남 